In [1]:
import datetime as dt
import math

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)

In [2]:
from datetime import date
from bisect import bisect_left

def get_expiry(input_date, index_symbol='nifty'):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_left(expiry_dates, input_date)    
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [3]:
# bnf_pandas = pd.read_csv("../data/bnf_min.csv")
bnf_pandas = pd.read_csv("../data/nifty_min.csv")
# bnf_pandas = pd.read_csv("../data/fin_min.csv")
# bnf_pandas = pd.read_csv("../data/midcp_min.csv")
# bnf_pandas = pd.read_csv("../data/sensex_min.csv")
# bnf_pandas = pd.read_csv("../data/bankex_min.csv")

In [4]:
bnf_pandas.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_pandas.head()

,index,datetime,o,h,l,c,v
0,nifty,2017-01-02 09:15:00,8210.1000,8211.7000,8189.0000,8189.5500,0
1,nifty,2017-01-02 09:16:00,8188.7500,8193.9500,8188.7500,8189.9500,0
2,nifty,2017-01-02 09:17:00,8190.1500,8190.7500,8173.7000,8173.7000,0
3,nifty,2017-01-02 09:18:00,8173.3500,8177.5500,8169.1500,8177.5500,0
4,nifty,2017-01-02 09:19:00,8177.8500,8178.1500,8173.4500,8174.4000,0


In [5]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2019]
# bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [6]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [7]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [8]:
# bnf = bnf.rename({"open": "o", "high": "h", "low": "l", "close": "c", "volume": "v"})

In [9]:
from expiries import dict_expiries

In [10]:
def resample(data, timeframe, offset=None):
    agg_list = [
        pl.col("o").first().alias("o"),
        pl.col("h").max().alias("h"),
        pl.col("l").min().alias("l"),
        pl.col("c").last().alias("c"),
    ]
    if timeframe == '10m':
        offset = '5m'
    if timeframe == '20m':
        offset = '15m'
    
    if "v" in data.columns:
        agg_list.append(pl.col("v").sum().alias("v"))
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(agg_list)
    )


In [64]:
import pandas as pd

def calculate_rsi(df: pd.DataFrame, period: int = 50) -> pd.Series:
    
    delta = df['c'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['rsi'] = rsi
    
    return df 


In [423]:
def generate_signals(df, roc_period = 1, period = 5, rsi_threshold = 70):
    
    df['daily_high_till_now'] = df.groupby(df['datetime'].dt.date)['h'].cummax()
    df['ROC'] = ((df['c'] - df['c'].shift(roc_period)) / df['c'].shift(roc_period)) * 100
    roc = df['ROC']
    
    df = calculate_rsi(df, period)
    # df['ma'] = df['c'].rolling(9).mean()
    
    # df['Sell Signal'] = (
    #     (roc.shift(2) >= 0) & 
    #     (roc.shift(1) >= 0) & 
    #     (roc < 0) &
    #     (
    #         (df['rsi'] > rsi_threshold) |
    #         (df['rsi'].shift(1) > rsi_threshold) |
    #         (df['rsi'].shift(2) > rsi_threshold)
    #     ) &
    #     (
    #         (df['c'] > df['ma']) &
    #         (df['c'].shift(1) > df['ma']) &
    #         (df['c'].shift(2) > df['ma'])
    #     )
    # ).astype(int)

    df['Sell Signal'] = (
        (roc.shift(2) >= 0) & 
        (roc.shift(1) >= 0) & 
        (roc < 0) &
        (
            (df['rsi'] > rsi_threshold) |
            (df['rsi'].shift(1) > rsi_threshold) |
            (df['rsi'].shift(2) > rsi_threshold)
        )
    ).astype(int)
    return df


In [424]:
data = bnf_pandas.copy()
# data['datetime'] = pd.to_datetime(data['datetime'].dt.date)
print(data["datetime"].tail())
trading_days_set = set(data["datetime"].dt.date)
# sorted(trading_days_set)

750760   2025-03-28 15:25:00
750761   2025-03-28 15:26:00
750762   2025-03-28 15:27:00
750763   2025-03-28 15:28:00
750764   2025-03-28 15:29:00
Name: datetime, dtype: datetime64[ns]


In [425]:
# GLOBAL VARIABLES

INSTRUMENT = "NIFTY"
INDEX = "nifty"
INDEX_MROUND = 50
# INDEX_MROUND=100

# INSTRUMENT = "NIFTY"
# INDEX = "nifty"
# INDEX_MROUND=50

PORTFOLIO_VALUE = 1_00_00_000
INDEX_LEV = 7
RPT_CE = 0.01
RPT_PE = 0.01
SLIPPAGE = 0.01
MAX_MARGIN = 100
# TF = "10m"

# SIGNAL_MA = 20
# NUM_OF_CANDELS = 1
# T_MA=20
# TARGET=100

In [426]:
bnf = resample(bnf, '1m')
data = bnf.to_pandas()
data.tail()
# data[['MA','signal_spot']]=MA(data,200)
# data[data['signal_spot']==1].head(50)
# bnf
# data.tail(50)
# data[data['datetime'].dt.date == dt.date(2024, 4, 29)]

,datetime,o,h,l,c,v
19783,2025-03-28 12:55:00,23526.6000,23527.5000,23471.1000,23493.9500,0
19784,2025-03-28 13:25:00,23493.6500,23522.5000,23453.6500,23473.2500,0
19785,2025-03-28 13:55:00,23472.5500,23574.7000,23451.3000,23571.8000,0
19786,2025-03-28 14:25:00,23573.2000,23612.0500,23540.0500,23591.5000,0
19787,2025-03-28 14:55:00,23591.1500,23591.2000,23450.2000,23495.1500,0


In [427]:
async def ce_trade(data, tf, offset, roc_period, spot_rsi_period, start_date, end_date):
    df_ = data.copy()
    df_polars = resample(pl.DataFrame(df_), tf, offset)
    df_ = df_polars.to_pandas()
    df = calculate_rsi(df_, spot_rsi_period)
    df['ma'] = df['c'].rolling(9).mean()
    # start_date = dt.date(2024, 1, 5)
    # end_date = dt.date(2025, 3, 31)
    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()
    time_of_day = dt.time(9, 15)
    trade_book = []
    ce_lowest_low = float("inf")
    ce_highest_high = float("-inf")
    entry_roc = 0
    opt_rsi = 0
    spot_rsi = 0
    is_trailing_active = False

    while current_date < end_date:
        print(f'CE : {current_date}')
        entry = 0
        initial_sl = 0
        exit = 0
        in_ce_trade = False
        in_pe_trade = False
        # signal_exist=False

        points_captured = 0
        remark = ""
        trailing_active = False
        tsl = 0
        stop_trading = False
        is_gap_ce_sl = False
        previous_ce_sl_hit = False
        current_date_increament_flag = False
        # tsl_high = 0

        starting_time = dt.time(9, 15)

        ending_time = dt.time(15, 30)

        if not in_ce_trade and current_date in trading_days_set:

            ce_search_datetime = dt.datetime.combine(current_date, time_of_day)
            # print(f'current date : {ce_search_datetime}')

            spot_open = df.loc[df["datetime"] >= ce_search_datetime, "o"].iloc[0]
            
            spot_atm = int(
                math.ceil(spot_open / INDEX_MROUND) * INDEX_MROUND
            )  ##ROUNDS TO NEAREST 500 OTM
            
            nearest_expiry = get_expiry(current_date)
            
            selected_strike_ce = spot_atm
            # print(f'selected strike CE : {selected_strike_ce}')
            ce_df = await fetch_data(
                index=INDEX,
                start_date=nearest_expiry - dt.timedelta(days=14),
                start_time=starting_time,
                end_date=nearest_expiry,
                end_time=ending_time,
                strike=selected_strike_ce,
                asset_class="C",
                expiry=nearest_expiry,
            )
            # print(ce_df)
            if ce_df is not None and not isinstance(ce_df, str):
                # print('new data fetched CE')
                data_ce = True
                ce_df = ce_df.select(["datetime", "o", "h", "l", "c", "v"])
                ce_df = resample(ce_df, tf, offset)
                ce_df_pandas = ce_df.to_pandas()
                ce_df_pandas = ce_df_pandas[ce_df_pandas['datetime'].dt.time != dt.time(15, 30)]
                ce_df = generate_signals(ce_df_pandas, roc_period)
                # ce_df = calculate_signals(ce_df_pandas)
                # print(ce_df.to_string())
            else:
                data_ce = False
                current_date += dt.timedelta(days=1)
                continue

            if data_ce:

                for i in range(0, len(ce_df)):
                    current_candle = ce_df.iloc[i]
                    current_candle_open = ce_df.iloc[i]["o"]
                    current_candle_high = ce_df.iloc[i]["h"]
                    current_candle_low = ce_df.iloc[i]["l"]
                    current_candle_close = ce_df.iloc[i]["c"]

                    previous_candle_low = ce_df.iloc[i - 1]["l"]
                    previous_candle_close = ce_df.iloc[i - 1]["c"]
                    

                    expiry = nearest_expiry
                    strike = selected_strike_ce
                    asset_class = "C"
                    # print(ce_df.iloc[i].to_string())

                    signal = (ce_df.iloc[i - 1]["Sell Signal"]) and (df.iloc[i-1]['c'] >= df.iloc[i-1]['ma']) 
                    # signal = (ce_df.iloc[i - 1]["Sell Signal"])
                    # candle_condition = (ce_df.iloc[i-1]['daily_high_till_now'] - previous_candle_close) > 10

                    if ce_df.iloc[i]["datetime"] >= ce_search_datetime:


                        if (
                            not previous_ce_sl_hit
                            and not in_ce_trade
                            and signal
                            # and current_candle_low < previous_candle_low
                            and ce_df.iloc[i]["datetime"].time() > time_of_day
                            and (
                                (nearest_expiry - ce_df.iloc[i]["datetime"].date()).days
                                >= 0
                                and (
                                    nearest_expiry - ce_df.iloc[i]["datetime"].date()
                                ).days
                                < 7
                            )
                            and ce_df.iloc[i]["datetime"].time() < dt.time(15, 15)
                            # and candle_condition
                        ):
                            # print(ce_df.iloc[i-1])
                            # print('####################################################################################################')
                            # print(f'entry found {current_candle_open}')
                            # print(f'entry datetime {ce_df.iloc[i-1]["datetime"]}')
                            # print('####################################################################################################')
                            
                            entry = current_candle_open
                            entry_date = ce_df.iloc[i-1]["datetime"].date()
                            entry_time = ce_df.iloc[i-1]["datetime"].time()
                            # initial_sl = ce_df.iloc[i - SL_CANDLES_NUM : i]["h"].max()
                            # initial_sl = entry * 1.3
                            initial_sl = ce_df.iloc[i-1]['daily_high_till_now']
                            in_ce_trade = True
                            ce_lowest_low = float("inf")
                            ce_highest_high = float("-inf")
                            # print(f'initial SL : {initial_sl}')
                            entry_roc = ce_df.iloc[i-1]['ROC']
                            opt_rsi = ce_df.iloc[i-1]['rsi']
                            spot_rsi = df.iloc[i-1]['rsi']
                            qty = RPT_CE * PORTFOLIO_VALUE / (initial_sl - entry)
                            if (
                                (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                            ) * 100 > MAX_MARGIN:
                                qty = PORTFOLIO_VALUE * INDEX_LEV / strike * (MAX_MARGIN / 100)
                            # qty = PORTFOLIO_VALUE * INDEX_LEV / spot_atm
                            # print(f'qty : {qty}')

                        # While in trade, track the highest high and lowest low
                        if in_ce_trade:
                            # Track the highest high
                            ce_highest_high = max(ce_highest_high, current_candle_high)

                            # Track the lowest low
                            ce_lowest_low = min(ce_lowest_low, current_candle_low)

                            # if ce_lowest_low < entry * (100 - decay) / 100:
                            #     is_trailing_active = True

                            # trailing_sl_signal = ce_df['Trailing Signal'].iloc[i]

                        if (
                            in_ce_trade
                            and ce_df.iloc[i]["datetime"].time() == dt.time(9, 15)
                            and current_candle_open > initial_sl
                        ):

                            exit = current_candle_close
                            in_ce_trade = False
                            stop_trading = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            remark = "Gap SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = ce_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = ce_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if in_ce_trade and current_candle_high > initial_sl:
                            exit = initial_sl
                            otm_datetime = ce_df.iloc[i]["datetime"]
                            in_ce_trade = False
                            stop_trading = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = ce_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = ce_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if (
                            in_ce_trade
                            and ce_df.iloc[i]["datetime"].date() == nearest_expiry
                            and ce_df.iloc[i]['datetime'].time() >= dt.time(15, 15)
                        ):
                            # print(ce_df.iloc[i])
                            # print(f'EOD exit {current_candle_close}')
                            # print(f'EOD datetime {ce_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            otm_datetime = ce_df.iloc[i]["datetime"]
                            in_ce_trade = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "EOD exit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = nearest_expiry + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            # print(f'current date increased by 1 on expiry : {current_date}')
                            is_trailing_active = False
                            break

                        if (
                            not in_ce_trade
                            and not previous_ce_sl_hit
                            and ce_df.iloc[i]["datetime"].time() >= dt.time(15, 15)
                        ):
                            # print('inside exoiry non trade date increment')
                            current_date = current_date + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            break

        if not current_date_increament_flag:
            current_date = current_date + dt.timedelta(days=1)
            current_date_increament_flag = False

    trade_book_df = pd.DataFrame(trade_book)

    return trade_book_df

In [428]:
async def pe_trade(data, tf, offset, roc_period, spot_rsi_period, start_date, end_date):
    df_ = data.copy()
    df_polars = resample(pl.DataFrame(df_), tf, offset)
    df_ = df_polars.to_pandas()
    df = calculate_rsi(df_, spot_rsi_period)
    df['ma'] = df['c'].rolling(9).mean()
    # start_date = dt.date(2024, 1, 5)
    # end_date = dt.date(2025, 3, 31)
    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()
    time_of_day = dt.time(9, 15)
    trade_book = []
    pe_lowest_low = float("inf")
    pe_highest_high = float("-inf")
    entry_roc = 0
    opt_rsi = 0
    spot_rsi = 0
    is_trailing_active = False

    while current_date < end_date:
        print(f'PE : {current_date}')
        entry = 0
        initial_sl = 0
        exit = 0
        in_pe_trade = False
        in_pe_trade = False
        # signal_exist=False

        points_captured = 0
        remark = ""
        trailing_active = False
        tsl = 0
        stop_trading = False
        is_gap_pe_sl = False
        previous_pe_sl_hit = False
        current_date_increament_flag = False
        # tsl_high = 0

        starting_time = dt.time(9, 15)

        ending_time = dt.time(15, 30)

        if not in_pe_trade and current_date in trading_days_set:

            pe_search_datetime = dt.datetime.combine(current_date, time_of_day)
            # print(f'current date : {pe_search_datetime}')

            spot_open = df.loc[df["datetime"] >= pe_search_datetime, "o"].iloc[0]
            # print(f'spot open : {spot_open}')
            # spot_atm = int(round(spot_open / INDEX_MROUND) * INDEX_MROUND)
            spot_atm = int(
                math.floor(spot_open / INDEX_MROUND) * INDEX_MROUND
            )  ##ROUNDS TO NEAREST 500 OTM
            # print(f'spot atm : {spot_atm}')
            # nearest_expiry = await get_expiry(current_date)
            nearest_expiry = get_expiry(current_date)
            # if current_date== nearest_expiry:
            #     next_expiry_passing_value = current_date + dt.timedelta(days=1)
            #     nearest_expiry = await get_expiry_nifty( next_expiry_passing_value)
            # print(f'passing date for expry : {current_date}')
            # nearest_expiry = await get_monthly_expiry_nifty(current_date)
            # print(f'nearest expiry{nearest_expiry}')
            selected_strike_pe = spot_atm
            # print(f'selected strike PE : {selected_strike_pe}')
            pe_df = await fetch_data(
                index=INDEX,
                start_date=nearest_expiry - dt.timedelta(days=14),
                start_time=starting_time,
                end_date=nearest_expiry,
                end_time=ending_time,
                strike=selected_strike_pe,
                asset_class="P",
                expiry=nearest_expiry,
            )
            if pe_df is not None and not isinstance(pe_df, str):
                # print('new data fetched PE')
                data_pe = True
                pe_df = pe_df.select(["datetime", "o", "h", "l", "c", "v"])
                pe_df = resample(pe_df, tf, offset)
                pe_df_pandas = pe_df.to_pandas()
                pe_df_pandas = pe_df_pandas[pe_df_pandas['datetime'].dt.time != dt.time(15, 30)]
                pe_df = generate_signals(pe_df_pandas, roc_period)
                # pe_df = calculate_signals(pe_df_pandas)
                # print(pe_df.to_string())
            else:
                data_pe = False
                current_date += dt.timedelta(days=1)
                continue

            if data_pe:

                for i in range(0, len(pe_df)):
                    current_candle = pe_df.iloc[i]
                    current_candle_open = pe_df.iloc[i]["o"]
                    current_candle_high = pe_df.iloc[i]["h"]
                    current_candle_low = pe_df.iloc[i]["l"]
                    current_candle_close = pe_df.iloc[i]["c"]

                    previous_candle_low = pe_df.iloc[i - 1]["l"]
                    previous_candle_close = pe_df.iloc[i - 1]["c"]
                    

                    expiry = nearest_expiry
                    strike = selected_strike_pe
                    asset_class = "P"
                    # print(pe_df.iloc[i].to_string())

                    signal = (pe_df.iloc[i - 1]["Sell Signal"]) and (df.iloc[i-1]['c'] <= df.iloc[i-1]['ma'])
                    # signal = (pe_df.iloc[i - 1]["Sell Signal"])
                    # candle_condition = (pe_df.iloc[i-1]['daily_high_till_now'] - previous_candle_close) > 10

                    if pe_df.iloc[i]["datetime"] >= pe_search_datetime:


                        if (
                            not previous_pe_sl_hit
                            and not in_pe_trade
                            and signal
                            # and current_candle_low < previous_candle_low
                            and pe_df.iloc[i]["datetime"].time() > time_of_day
                            and (
                                (nearest_expiry - pe_df.iloc[i]["datetime"].date()).days
                                >= 0
                                and (
                                    nearest_expiry - pe_df.iloc[i]["datetime"].date()
                                ).days
                                < 7
                            )
                            and pe_df.iloc[i]["datetime"].time() < dt.time(15, 15)
                            # and candle_condition
                        ):
                            # print(pe_df.iloc[i-1])
                            # print('####################################################################################################')
                            # print(f'entry found {current_candle_open}')
                            # print(f'entry datetime {pe_df.iloc[i-1]["datetime"]}')
                            # print('####################################################################################################')
                            
                            # today_data = pe_df[pe_df['datetime'].dt.date == current_candle['datetime'].date()]
                            # day_high = today_data.iloc[0 : i]['h'].max()
                            # print(today_data.to_string())
                            
                            entry = current_candle_open
                            entry_date = pe_df.iloc[i-1]["datetime"].date()
                            entry_time = pe_df.iloc[i-1]["datetime"].time()
                            # initial_sl = pe_df.iloc[i - SL_CANDLES_NUM : i]["h"].max()
                            # initial_sl = entry * 1.3
                            initial_sl = pe_df.iloc[i-1]['daily_high_till_now']
                            in_pe_trade = True
                            pe_lowest_low = float("inf")
                            pe_highest_high = float("-inf")
                            entry_roc = pe_df.iloc[i-1]['ROC']
                            opt_rsi = pe_df.iloc[i-1]['rsi']
                            spot_rsi = df.iloc[i-1]['rsi']
                            qty = RPT_PE * PORTFOLIO_VALUE / (initial_sl - entry)
                            if (
                                (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                            ) * 100 > MAX_MARGIN:
                                qty = PORTFOLIO_VALUE * INDEX_LEV / strike * (MAX_MARGIN / 100)

                            # qty = PORTFOLIO_VALUE * INDEX_LEV / spot_atm

                        # While in trade, track the highest high and lowest low
                        if in_pe_trade:
                            # Track the highest high
                            pe_highest_high = max(pe_highest_high, current_candle_high)

                            # Track the lowest low
                            pe_lowest_low = min(pe_lowest_low, current_candle_low)
                            
                            # if pe_lowest_low < entry * (100 - decay) / 100:
                            #     is_trailing_active = True

                            # trailing_sl_signal = pe_df['Trailing Signal'].iloc[i]
                           
                        if (
                            in_pe_trade
                            and pe_df.iloc[i]["datetime"].time() == dt.time(9, 15)
                            and current_candle_open > initial_sl
                        ):

                            # print(pe_df.iloc[i])
                            # print(f'GAP sl hit {initial_sl}')
                            # print(f'GAP sl datetime {pe_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            in_pe_trade = False
                            stop_trading = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            remark = "Gap SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = pe_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = pe_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if in_pe_trade and current_candle_high > initial_sl:
                            # print(pe_df.iloc[i])
                            # print(f'initial sl hit {initial_sl}')
                            # print(f'initial sl datetime {pe_df.iloc[i]["datetime"]}')
                            exit = initial_sl
                            otm_datetime = pe_df.iloc[i]["datetime"]
                            in_pe_trade = False
                            stop_trading = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            tsl_high = 0
                            points_captured = 0
                            current_date = pe_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = pe_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if (
                            in_pe_trade
                            and pe_df.iloc[i]["datetime"].date() == nearest_expiry
                            and pe_df.iloc[i]['datetime'].time() >= dt.time(15, 15)
                        ):
                            # print(pe_df.iloc[i])
                            # print(f'EOD exit {current_candle_close}')
                            # print(f'EOD datetime {pe_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            otm_datetime = pe_df.iloc[i]["datetime"]
                            in_pe_trade = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "EOD exit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = nearest_expiry + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            # print(f'current date increased by 1 on expiry : {current_date}')
                            is_trailing_active = False
                            break

                        if (
                            not in_pe_trade
                            and not previous_pe_sl_hit
                            and pe_df.iloc[i]["datetime"].time() >= dt.time(15, 15)
                        ):
                            # print('inside exoiry non trade date increment')
                            current_date = current_date + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            break

        if not current_date_increament_flag:
            current_date = current_date + dt.timedelta(days=1)
            current_date_increament_flag = False

    trade_book_df = pd.DataFrame(trade_book)

    return trade_book_df

In [429]:
# tb_ce = pd.DataFrame()
# tb_pe = pd.DataFrame()

async def execute(DF, tf, offset, roc_period, spot_rsi_period, start_dt, end_dt):
    data = DF.copy()
    tb_ce = await ce_trade(data, tf, offset, roc_period, spot_rsi_period, start_dt, end_dt)
    tb_pe = await pe_trade(data, tf, offset, roc_period, spot_rsi_period, start_dt, end_dt)
    tb = pd.concat([tb_ce, tb_pe], ignore_index=True)
    # print(len(tb))
    if len(tb)>0:
        tb = tb.sort_values(by="date")
    return tb

In [430]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [449]:
tf1 = '10m'
offset1 = '5m'
roc_period_ = 4
spot_rsi_period_ = 11

start_date = dt.date(2024, 1, 1)
end_date = dt.date(2024, 12, 31)

bnf = resample(bnf, tf1)
data = bnf.to_pandas()
tb = await execute(data, tf1, offset1, roc_period_, spot_rsi_period_, start_date, end_date)

CE : 2024-01-01
CE : 2024-01-01
CE : 2024-01-02
CE : 2024-01-05
CE : 2024-01-12
CE : 2024-01-13
CE : 2024-01-14
CE : 2024-01-15
CE : 2024-01-15
CE : 2024-01-16
CE : 2024-01-16
CE : 2024-01-17
CE : 2024-01-18
CE : 2024-01-19
CE : 2024-01-20
CE : 2024-01-26
CE : 2024-01-27
CE : 2024-01-28
CE : 2024-01-29
CE : 2024-01-29
CE : 2024-01-30
CE : 2024-01-31
CE : 2024-02-01
CE : 2024-02-02
CE : 2024-02-09
CE : 2024-02-10
CE : 2024-02-11
CE : 2024-02-12
CE : 2024-02-16
CE : 2024-02-19
CE : 2024-02-19
CE : 2024-02-23
CE : 2024-02-23
CE : 2024-03-01
CE : 2024-03-01
CE : 2024-03-02
CE : 2024-03-08
CE : 2024-03-09
CE : 2024-03-10
CE : 2024-03-11
CE : 2024-03-12
CE : 2024-03-15
CE : 2024-03-16
CE : 2024-03-17
CE : 2024-03-18
CE : 2024-03-18
CE : 2024-03-19
CE : 2024-03-20
CE : 2024-03-21
CE : 2024-03-22
CE : 2024-03-22
CE : 2024-03-23
CE : 2024-03-24
CE : 2024-03-25
CE : 2024-03-26
CE : 2024-03-27
CE : 2024-03-28
CE : 2024-03-28
CE : 2024-03-29
CE : 2024-03-30
CE : 2024-03-31
CE : 2024-04-01
CE : 202

In [450]:
tb['DATETIME'] = pd.to_datetime(tb['date'].astype(str) + ' ' + tb['Entry Time'].astype(str))

In [451]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, 'Kitkat Base')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

/tmp/ipykernel_1648/3277872145.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2023,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2024,2.5135,227,30.3965,2.1573,-0.9262,-14.9391,0.1683,Kitkat Base
2025,-1.0510,1,0.0000,NaN,-1.0510,0.0000,-inf,Kitkat Base
Overall,1.4625,228,30.2632,2.1573,-0.9270,-15.9901,0.0915,Kitkat Base


In [405]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, 'Kitkat Base')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

/tmp/ipykernel_1648/3277872145.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2023,0.0000,0,NaN,NaN,NaN,NaN,NaN,Kitkat Base
2024,28.9804,292,27.0548,3.0493,-0.9949,-19.2929,1.5021,Kitkat Base
2025,-2.1255,1,0.0000,NaN,-2.1255,0.0000,-inf,Kitkat Base
Overall,26.8549,293,26.9625,3.0493,-1.0002,-19.2929,1.3920,Kitkat Base


In [391]:
tb

,date,day,expiry,DTE,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,ROC on Entry,Opt RSI on Entry,Spot RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME
36,2024-01-05,Friday,2024-01-18,13,21700,P,164.1500,11:45:00,186.7000,186.7000,2024-01-05,13:45:00,-6.7610,59.4228,66.3405,SL hit,-22.5500,3.5085,3225.8065,-84059.6774,-0.8406,2024,1,196.3000,153.7000,0.3371,100.0000,2024-01-05 11:45:00
37,2024-01-05,Friday,2024-01-18,13,21650,P,154.4000,14:15:00,186.0000,186.0000,2024-01-08,10:45:00,-9.3195,63.1579,9.6054,SL hit,-31.6000,3.4040,3233.2564,-113176.9053,-1.1318,2024,1,186.2500,115.0000,1.2739,100.0000,2024-01-05 14:15:00
0,2024-01-12,Friday,2024-01-25,13,21750,C,270.5000,10:45:00,294.3000,294.3000,2024-01-12,12:45:00,-1.6235,93.3183,65.3868,SL hit,-23.8000,5.6480,3218.3908,-94775.1724,-0.9478,2024,1,305.8000,260.5000,0.3218,100.0000,2024-01-12 10:45:00
1,2024-01-19,Friday,2024-02-01,13,21700,C,217.4500,10:45:00,266.9000,260.0000,2024-01-20,09:15:00,-13.0678,18.4769,65.3868,Gap SL hit,-42.5500,4.7745,3225.8065,-152659.6774,-1.5266,2024,1,290.0000,212.4000,0.1629,100.0000,2024-01-19 10:45:00
38,2024-01-20,Saturday,2024-02-01,12,21650,P,182.4000,12:45:00,210.0000,210.0000,2024-01-20,14:45:00,-9.2776,57.6749,22.8468,SL hit,-27.6000,3.9240,3233.2564,-101925.1732,-1.0193,2024,1,230.2000,178.9000,0.1132,100.0000,2024-01-20 12:45:00
2,2024-01-29,Monday,2024-02-08,10,21550,C,356.2000,11:15:00,365.2500,365.2500,2024-01-29,11:45:00,-2.6003,55.2885,63.6596,SL hit,-9.0500,7.2145,3248.2599,-52831.3225,-0.5283,2024,1,370.0000,354.0000,0.0715,100.0000,2024-01-29 11:15:00
3,2024-02-02,Friday,2024-02-15,13,21950,C,258.0500,10:15:00,275.0000,275.0000,2024-02-02,10:45:00,-4.2945,92.0014,64.8097,SL hit,-16.9500,5.3305,3189.0661,-71053.9863,-0.7105,2024,2,298.8000,254.0000,0.1292,100.0000,2024-02-02 10:15:00
39,2024-02-02,Friday,2024-02-15,13,21900,P,200.3500,13:45:00,250.0500,250.0500,2024-02-05,09:15:00,-11.2517,58.9474,12.1100,SL hit,-49.7000,4.5040,3196.3470,-173254.7945,-1.7325,2024,2,256.0000,187.9000,0.3979,100.0000,2024-02-02 13:45:00
4,2024-02-05,Monday,2024-02-15,10,21950,C,222.2000,10:15:00,264.5500,264.5500,2024-02-07,09:15:00,-7.0231,64.2094,39.8046,SL hit,-42.3500,4.8675,3189.0661,-150579.7267,-1.5058,2024,2,275.2000,129.0000,2.9722,100.0000,2024-02-05 10:15:00
5,2024-02-09,Friday,2024-02-22,13,21800,C,242.7000,09:15:00,260.2000,260.2000,2024-02-15,09:15:00,-6.2984,44.1581,61.1144,SL hit,-17.5000,5.0290,3211.0092,-72340.8257,-0.7234,2024,2,272.2000,96.0500,4.7089,100.0000,2024-02-09 09:15:00


In [338]:
tb_eod = tb[tb['Remark'] == 'EOD exit']
tb_eod['Entry Price'].mean()
# tb['Points Captured'].min()

123.31509433962263

In [297]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

/tmp/ipykernel_1648/3277872145.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2023,0.0000,0,NaN,NaN,NaN,NaN,NaN,...
2024,-18.3699,145,20.6897,3.3246,-1.0270,-45.6728,-0.4022,...
2025,-0.9854,1,0.0000,NaN,-0.9854,0.0000,-inf,...
Overall,-19.3552,146,20.5479,3.3246,-1.0267,-45.6728,-0.4238,...


# Moving Parts : 
## ROC period
## TF
## No. of entries per week
## RSI (optional)
## MA for Spot / Options - Both
## SL -> Day high, Prev Candle High, x% from entry, etc.
## Targets

In [96]:
# tb.to_csv('Hammer_10m_15_50.csv', index=False)

In [97]:
weekly_roi = tb.groupby('expiry')['ROI%'].sum().reset_index()
weekly_roi['expiry'] = pd.to_datetime(weekly_roi['expiry'])
weekly_roi['Trade Year'] = weekly_roi['expiry'].dt.year
weekly_roi

,expiry,ROI%,Trade Year
0,2019-01-31,5.3868,2019
1,2019-02-14,0.7503,2019
2,2019-02-21,5.6888,2019
3,2019-02-28,-7.3821,2019
4,2019-03-07,6.1056,2019
5,2019-03-14,-0.8796,2019
6,2019-03-20,1.6605,2019
7,2019-03-28,16.5614,2019
8,2019-04-04,17.4322,2019
9,2019-04-11,-0.3063,2019


In [98]:
stats = generate_stats(weekly_roi, 'Hammer Weekly')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,165.8776,47,70.2128,6.4200,-3.2845,-8.9285,18.5785,Hammer Weekly
2020,113.7935,53,62.2642,5.4292,-3.2685,-14.4217,7.8904,Hammer Weekly
2021,111.9018,52,71.1538,4.5305,-3.7151,-8.7373,12.8074,Hammer Weekly
2022,108.0109,52,53.8462,7.4428,-4.1828,-16.9439,6.3746,Hammer Weekly
2023,78.0217,51,58.8235,4.4648,-2.6630,-13.4325,5.8084,Hammer Weekly
2024,84.3846,52,57.6923,5.0805,-3.0923,-17.8927,4.7161,Hammer Weekly
2025,27.0750,14,57.1429,5.4726,-2.7843,-7.5346,3.5934,Hammer Weekly
Overall,689.0651,321,61.9938,5.5135,-3.3453,-21.4835,32.0742,Hammer Weekly


In [99]:
tb.tail(25)

,date,day,expiry,DTE,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME
1273,2025-02-10,Monday,2025-02-13,3,23500,P,205.6000,14:25:00,234.0000,234.0000,2025-02-11,09:15:00,42.9266,SL hit,-28.4000,4.3960,704.2254,-23095.7746,-2.3096,2025,2,245.4500,169.9000,2.5141,206.8662,2025-02-10 14:25:00
1274,2025-02-12,Wednesday,2025-02-13,1,23050,P,97.0000,13:45:00,255.7000,18.6000,2025-02-13,15:25:00,35.7422,EOD exit,78.4000,1.1560,126.0239,9734.5936,0.9735,2025,2,137.4000,2.4000,1.1922,36.3106,2025-02-12 13:45:00
644,2025-02-14,Friday,2025-02-20,6,23100,C,162.6000,09:25:00,205.9000,0.0500,2025-02-20,15:25:00,30.8669,EOD exit,162.5500,1.6265,461.8938,74329.5612,7.4330,2025,2,187.8500,0.0500,7.5081,133.3718,2025-02-14 09:25:00
1275,2025-02-14,Friday,2025-02-20,6,23050,P,144.9500,09:55:00,186.3500,186.3500,2025-02-14,10:35:00,45.6183,SL hit,-41.4000,3.3130,483.0918,-21600.4831,-2.1600,2025,2,236.1000,130.4500,0.7005,139.1908,2025-02-14 09:55:00
1276,2025-02-14,Friday,2025-02-20,6,23000,P,197.2000,14:25:00,287.5500,287.5500,2025-02-17,09:15:00,41.9406,SL hit,-90.3500,4.8475,221.3614,-21073.0493,-2.1073,2025,2,297.9000,160.6000,0.8102,63.6414,2025-02-14 14:25:00
1277,2025-02-17,Monday,2025-02-20,3,22800,P,128.8000,10:25:00,181.8000,0.0500,2025-02-20,15:25:00,46.9944,EOD exit,128.7500,1.2885,377.3585,48098.6792,4.8099,2025,2,148.3500,0.0500,4.8585,107.5472,2025-02-17 10:25:00
645,2025-02-21,Friday,2025-02-27,6,22900,C,172.2000,09:15:00,183.4000,0.1000,2025-02-27,15:25:00,46.8366,EOD exit,172.1000,1.7230,873.3624,148800.8734,14.8801,2025,2,174.9000,0.0500,15.0349,250.0000,2025-02-21 09:15:00
1278,2025-02-21,Friday,2025-02-27,6,22850,P,125.0000,09:15:00,150.0000,150.0000,2025-02-21,09:45:00,34.3750,SL hit,-25.0000,2.7500,800.0000,-22200.0000,-2.2200,2025,2,159.7500,117.3000,0.6160,228.5000,2025-02-21 09:15:00
1279,2025-02-21,Friday,2025-02-27,6,22800,P,134.5000,12:15:00,182.1500,216.4000,2025-02-24,09:15:00,49.7712,Gap SL hit,-81.9000,3.5090,419.7272,-35848.4785,-3.5848,2025,2,225.6000,119.8500,0.6149,119.6222,2025-02-21 12:15:00
1280,2025-02-24,Monday,2025-02-27,3,22600,P,99.5500,12:25:00,131.0000,54.7500,2025-02-27,15:25:00,41.7171,EOD exit,44.8000,1.5430,635.9300,27508.4261,2.7508,2025,2,124.1500,26.6500,4.6359,179.6502,2025-02-24 12:25:00


In [28]:
tb_ce = tb[tb['type'] == 'C']
tb_pe = tb[tb['type'] == 'P']

In [29]:
# tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb_ce, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,61.3045,112,33.9286,5.3942,-1.9415,-26.8144,2.2863,...
2020,36.9525,128,33.5938,5.1875,-2.1895,-62.3433,0.5927,...
2021,18.5655,122,33.6066,4.8782,-2.2400,-27.6852,0.6706,...
2022,28.9205,119,30.2521,6.3562,-2.4085,-33.3810,0.8664,...
2023,18.3090,121,29.7521,5.0317,-1.9157,-41.2694,0.4436,...
2024,45.7616,110,32.7273,5.5123,-2.0633,-48.4864,0.9438,...
2025,43.8320,23,43.4783,7.2385,-2.1964,-11.6965,3.7475,...
Overall,253.6456,735,32.6531,5.4535,-2.1317,-70.7034,3.5875,...


In [30]:
# tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb_pe, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,69.6642,109,35.7798,5.4556,-2.0444,-24.7296,2.8170,...
2020,114.0508,116,38.7931,6.3034,-2.3888,-39.8218,2.8640,...
2021,64.5999,107,40.1869,4.8307,-2.2363,-19.9513,3.2379,...
2022,90.7670,127,34.6457,6.4393,-2.3200,-48.0643,1.8884,...
2023,42.1746,117,33.3333,5.0981,-2.0083,-35.0449,1.2034,...
2024,8.5159,110,32.7273,4.9054,-2.2713,-47.5886,0.1789,...
2025,-7.1063,37,35.1351,3.4161,-2.1465,-16.6179,-0.4276,...
Overall,382.6661,723,35.8230,5.4336,-2.2083,-54.7600,6.9881,...


In [27]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,130.9686,221,34.8416,5.4253,-1.9915,-23.6824,5.5302,...
2020,151.0034,244,36.0656,5.7581,-2.2802,-26.4993,5.6984,...
2021,83.1654,229,36.6812,4.8539,-2.2384,-22.4713,3.7010,...
2022,119.6875,246,32.5203,6.4019,-2.3642,-28.1178,4.2566,...
2023,60.4836,238,31.5126,5.0662,-1.9600,-20.8403,2.9022,...
2024,58.5436,218,33.0275,5.2089,-2.1678,-25.1252,2.3301,...
2025,44.2623,45,37.7778,6.1039,-2.1252,-11.1397,3.9734,...
Overall,648.1145,1441,34.2124,5.4830,-2.1677,-28.1178,23.0500,...


In [26]:
tb_margin = tb[tb['Margin'] > 100]
tb_margin['ROI%'].sum(), len(tb_margin), len(tb)

(534.3461430674945, 1053, 1458)

In [25]:
# tb.to_csv('EMAR_5m_50EMA.csv', index=False)

In [27]:
# tb.tail(50)

In [37]:
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,113.1501,230,34.3478,5.0277,-1.8810,-22.5788,5.0113,...
2020,134.9478,256,34.3750,5.8342,-2.2527,-35.2967,3.8232,...
2021,97.6609,233,36.0515,4.9230,-2.1199,-21.8114,4.4775,...
2022,103.6360,248,31.8548,6.3140,-2.3383,-36.9616,2.8039,...
2023,119.8259,217,35.9447,4.9810,-1.9330,-14.0168,8.5487,...
2024,45.9805,231,32.0346,4.9413,-2.0362,-31.7348,1.4489,...
2025,46.0862,44,38.6364,5.9487,-2.0386,-8.8266,5.2213,...
Overall,661.2874,1459,34.2015,5.3672,-2.1010,-36.9616,17.8912,...


# Simulation

In [ ]:
TF_ = ['10m']
EMA_ = [18]
RSI_ = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
# MA_Range = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
RSI_THRESHOLD_ = [30, 40, 50, 60, 70, 80]

for tf in TF_:
    for ema in EMA_:
        for rsi in RSI_:
            for rsi_threshold in RSI_THRESHOLD_:
                variation = f'5m, 50EMA, RSI: {rsi}, RSI Threshold: {rsi_threshold}'
                print(variation)
                if tf == '10m':
                    offset = '5m'
                elif tf >= '20m':
                    offset = '15m'
                else:
                    offset = '0m'
                bnfx = resample(bnf, tf, offset)
                data = bnfx.to_pandas()
                tb_ce = await ce_trade(data, tf, offset, ema, rsi, rsi_threshold)
                tb_pe = await pe_trade(data, tf, offset, ema, rsi, rsi_threshold)
                tb = pd.concat([tb_ce, tb_pe], ignore_index=True)

                if len(tb) > 0:
                    tb = tb.sort_values(by="date")
                    tb['DATETIME'] = pd.to_datetime(tb['date'].astype(str) + ' ' + tb['Entry Time'].astype(str))
                    tb = tb.sort_values(by='DATETIME')
                    stats = generate_stats(tb, variation)
                    for x, y in stats.items():
                        z = pd.DataFrame(y)
                        print(z.to_string())

                        

5m, 50EMA, RSI: 5, RSI Threshold: 30
